In [66]:
import re
import json
import pandas as pd
import geopandas as gpd

In [8]:
pd.set_option( 'display.max_columns', None )
pd.set_option( 'display.max_rows', None )

In [81]:
localidades = gpd.read_file( './data/geo/localidades_no_sumapaz.json' )

In [82]:
del localidades[ 'geometry' ]

In [83]:
localidades.head()

,localidad,codigo_localidad
0,Antonio Nariño,15
1,Tunjuelito,6
2,Rafael Uribe Uribe,18
3,La Candelaria,17
4,Barrios Unidos,12


In [84]:
locs = {}
for i, row in localidades.iterrows():
    locs[ str( row[ 'codigo_localidad' ] ) ] = row[ 'localidad' ]
locs

{'15': 'Antonio Nariño',
 '6': 'Tunjuelito',
 '18': 'Rafael Uribe Uribe',
 '17': 'La Candelaria',
 '12': 'Barrios Unidos',
 '13': 'Teusaquillo',
 '16': 'Puente Aranda',
 '14': 'Los Mártires',
 '1': 'Usaquén',
 '2': 'Chapinero',
 '3': 'Santa Fé',
 '4': 'San Cristóbal',
 '5': 'Usme',
 '19': 'Ciudad Bolívar',
 '7': 'Bosa',
 '8': 'Kennedy',
 '9': 'Fontibón',
 '10': 'Engativá',
 '11': 'Suba'}

In [85]:
agents = pd.read_excel( './data/agents_bogota.xlsx' )

In [86]:
agents.shape

(200, 24)

In [87]:
agents.dtypes

Susceptibles                   int64
Total infectados               int64
Graves                         int64
Críticos                       int64
Leves                          int64
Recuperados                    int64
Rt                           float64
Recuento_zonas                object
0-4                           object
5-19                          object
20-39                         object
40-59                         object
>60                           object
En_cuarentena                 object
Vivos                          int64
Día                            int64
Contactos_prom_trabajo       float64
Contactos_prom_transporte    float64
Contactos_prom_casa          float64
Nuevos_infectados              int64
Nuevos_infectados:acum         int64
Tasa de ataque               float64
Detectados                     int64
En_testing                     int64
dtype: object

In [127]:
agents.head( 5 )

,Susceptibles,Total infectados,Graves,Críticos,Leves,Recuperados,Rt,Vivos,Día,Contactos_prom_trabajo,Contactos_prom_transporte,Contactos_prom_casa,Nuevos_infectados,Nuevos_infectados:acum,Tasa de ataque,Detectados,En_testing
0,45151,42,0,0,42,0,0.000000,45193,0,0.000000,0.000000,0.000000,0,0,0.000000,0,0
1,45118,75,4,1,70,0,0.440000,45193,1,6.170203,10.804572,7.073927,17,17,0.000376,0,5
2,45096,96,5,3,88,0,0.572917,45192,2,5.898234,10.710812,6.513564,23,40,0.000885,0,8
3,45057,135,6,7,122,0,0.696296,45192,3,5.784542,10.653766,6.313706,30,70,0.001549,0,13
4,45038,153,6,6,141,0,0.732026,45191,4,5.698745,10.561926,6.194818,35,105,0.002323,0,12


In [126]:
agents.tail( 5 )

,Susceptibles,Total infectados,Graves,Críticos,Leves,Recuperados,Rt,Vivos,Día,Contactos_prom_trabajo,Contactos_prom_transporte,Contactos_prom_casa,Nuevos_infectados,Nuevos_infectados:acum,Tasa de ataque,Detectados,En_testing
195,12343,386,46,29,311,29209,0.194301,41938,195,2.460942,5.691688,2.605179,12,32800,0.725776,8433,53
196,12325,348,47,28,273,29260,0.204023,41933,196,2.452627,5.681826,2.613813,14,32814,0.726086,8442,51
197,12313,320,41,27,252,29296,0.206250,41929,197,2.452455,5.687973,2.617759,20,32834,0.726528,8445,56
198,12301,294,33,27,234,29331,0.197279,41926,198,2.450961,5.690049,2.619091,10,32844,0.726750,8448,57
199,12293,277,34,26,217,29353,0.187726,41923,199,2.459843,5.716623,2.611216,8,32852,0.726927,8456,49


In [89]:
recuento_zonas = { 0: 'Susceptibles', 3: 'Leves', 6: 'Graves', 9: 'Críticos', 12: 'Recuperados'  }

In [90]:
def expand_by_loc( x ):
    replacer = re.compile("(\d+)")
    x[ 'Recuento_zonas' ] = replacer.sub( r'"\1"', x[ 'Recuento_zonas' ] )
    x[ 'Recuento_zonas' ] = json.loads( x[ 'Recuento_zonas' ] )
    ds = []
    for z in x[ 'Recuento_zonas' ].keys():
        d = dict()
        d[ 'Localidad' ] = locs[ z ]
        d[ 'Día' ] = row[ 'Día' ]
        for i in recuento_zonas.keys():
            d[ recuento_zonas[ i ] ] = int( x[ 'Recuento_zonas' ][ z ][ i ] )
        ds.append( d )
    return ds

all_locs = []
for i, row in agents.iterrows():
    all_locs = all_locs + expand_by_loc( row )

all_locs = pd.DataFrame( all_locs )

In [92]:
all_locs.head()

,Localidad,Día,Susceptibles,Leves,Graves,Críticos,Recuperados
0,Kennedy,0,6868,9,0,0,0
1,Suba,0,7371,8,0,0,0
2,Rafael Uribe Uribe,0,1870,3,0,0,0
3,Engativá,0,4758,4,0,0,0
4,San Cristóbal,0,2084,1,0,0,0


In [121]:
all_locs.to_csv( './data/agents/locs_bogota.csv', index = False )

In [93]:
grupo_edad = { 0: 'Susceptibles', 1: 'Leves', 2: 'Graves', 3: 'Críticos', 4: 'Recuperados'  }

In [101]:
def expand_by_age( x ):
    ds = []
    for a in [ '0-4', '5-19', '20-39', '40-59', '>60' ]:
        x[ a ] = json.loads( x[ a ] )
        d = dict()
        d[ 'Grupo de edad' ] = a
        d[ 'Día' ] = row[ 'Día' ]
        for i in range( 5 ):
            d[ grupo_edad[ i ] ] = x[ a ][ i ]
        ds.append( d )
    return ds

all_ages = []
for i, row in agents.iterrows():
    all_ages = all_ages + expand_by_age( row )

all_ages = pd.DataFrame( all_ages )

In [102]:
all_ages.tail()

,Grupo de edad,Día,Susceptibles,Leves,Graves,Críticos,Recuperados
995,0-4,199,928,8,4,5,878
996,5-19,199,2518,34,0,7,5414
997,20-39,199,4317,92,10,1,11260
998,40-59,199,3278,64,17,1,8943
999,>60,199,1252,19,3,12,2858


In [118]:
all_ages.to_csv( './data/agents/ages_bogota.csv', index = False )

In [108]:
quarentenas = { 0: 'Susceptibles', 1: 'Leves', 2: 'Graves', 3: 'Críticos', 4: 'Recuperados'  }

In [109]:
def expand_by_quarintine( x ):
    ds = []
    x[ 'En_cuarentena' ] = json.loads( x[ 'En_cuarentena' ] )
    d = dict()
    d[ 'Día' ] = row[ 'Día' ]
    for i in range( 5 ):
        d[ quarentenas[ i ] ] = x[ 'En_cuarentena' ][ i ]
    ds.append( d )
    return ds

all_quarintines = []
for i, row in agents.iterrows():
    all_quarintines = all_quarintines + expand_by_quarintine( row )

all_quarintines = pd.DataFrame( all_quarintines )

In [111]:
all_quarintines.tail()

,Día,Susceptibles,Leves,Graves,Críticos,Recuperados
195,195,4189,84,27,27,12202
196,196,4185,76,24,25,12218
197,197,4183,64,18,23,12228
198,198,4177,57,13,24,12236
199,199,4175,52,13,22,12235


In [117]:
all_quarintines.to_csv( './data/agents/quarantine_bogota.csv', index = False )

In [105]:
del agents[ 'Recuento_zonas' ]
del agents[ '0-4' ]
del agents[ '5-19' ]
del agents[ '20-39' ]
del agents[ '40-59' ]
del agents[ '>60' ]
del agents[ 'En_cuarentena' ]

In [123]:
agents.tail()

,Susceptibles,Total infectados,Graves,Críticos,Leves,Recuperados,Rt,Vivos,Día,Contactos_prom_trabajo,Contactos_prom_transporte,Contactos_prom_casa,Nuevos_infectados,Nuevos_infectados:acum,Tasa de ataque,Detectados,En_testing
195,12343,386,46,29,311,29209,0.194301,41938,195,2.460942,5.691688,2.605179,12,32800,0.725776,8433,53
196,12325,348,47,28,273,29260,0.204023,41933,196,2.452627,5.681826,2.613813,14,32814,0.726086,8442,51
197,12313,320,41,27,252,29296,0.206250,41929,197,2.452455,5.687973,2.617759,20,32834,0.726528,8445,56
198,12301,294,33,27,234,29331,0.197279,41926,198,2.450961,5.690049,2.619091,10,32844,0.726750,8448,57
199,12293,277,34,26,217,29353,0.187726,41923,199,2.459843,5.716623,2.611216,8,32852,0.726927,8456,49


In [116]:
agents.dtypes

Susceptibles                   int64
Total infectados               int64
Graves                         int64
Críticos                       int64
Leves                          int64
Recuperados                    int64
Rt                           float64
Vivos                          int64
Día                            int64
Contactos_prom_trabajo       float64
Contactos_prom_transporte    float64
Contactos_prom_casa          float64
Nuevos_infectados              int64
Nuevos_infectados:acum         int64
Tasa de ataque               float64
Detectados                     int64
En_testing                     int64
dtype: object

In [115]:
agents.to_csv( './data/agents/agents_bogota.csv', index = False )